<a href="https://colab.research.google.com/github/juneilsam/TensorFlowDeveloper/blob/main/Exercises/Exercise_4_Handling_Complex_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[원작자 깃헙](https://github.com/lmoroney/dlaicourse)

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import tensorflow as tf
import os
import zipfile

In [2]:
DESIRED_ACCURACY = 0.999

In [6]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

--2021-11-28 11:29:25--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.97.128, 172.217.193.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2021-11-28 11:29:25 (178 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [13]:
zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", "r")
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [8]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>DESIRED_ACCURACY):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

In [9]:
callbacks = myCallback()

In [10]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
from tensorflow.keras.optimizers import RMSprop

In [19]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    "/tmp/h-or-s",
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.


In [20]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/15
8/8 [==============================] - 3s 238ms/step - loss: 1.5937 - accuracy: 0.6250
Epoch 2/15
8/8 [==============================] - 2s 228ms/step - loss: 0.4610 - accuracy: 0.7500
Epoch 3/15
8/8 [==============================] - 2s 229ms/step - loss: 0.2578 - accuracy: 0.8750
Epoch 4/15
8/8 [==============================] - 2s 228ms/step - loss: 0.2306 - accuracy: 0.9125
Epoch 5/15
8/8 [==============================] - 2s 231ms/step - loss: 0.1042 - accuracy: 0.9625
Epoch 6/15
8/8 [==============================] - 2s 228ms/step - loss: 0.1718 - accuracy: 0.9250
Epoch 7/15
8/8 [==============================] - 2s 231ms/step - loss: 0.0615 - accuracy: 0.9625
Epoch 8/15
8/8 [==============================] - 2s 229ms/step - loss: 0.0358 - accuracy: 0.9875
Epoch 9/15
8/8 [==============================] - 2s 229ms/step - loss: 0.0262 - accuracy: 0.9875
Epoch 10/15
8/8 [==============================] - 2s 228ms/step - loss: 0.0323 - accuracy: 0.9875
Epoch 11/15
8/8 [==

In [23]:
history.history

{'accuracy': [0.625,
  0.75,
  0.875,
  0.9125000238418579,
  0.9624999761581421,
  0.925000011920929,
  0.9624999761581421,
  0.987500011920929,
  0.987500011920929,
  0.987500011920929,
  1.0],
 'loss': [1.5936660766601562,
  0.4609639644622803,
  0.2577721178531647,
  0.23063747584819794,
  0.10419225692749023,
  0.17179623246192932,
  0.06151462346315384,
  0.035834766924381256,
  0.026156341657042503,
  0.0322665199637413,
  0.0024997168220579624]}